In [1]:
import numpy as np
from math import sqrt

In [223]:
class jTensor(np.ndarray):
    def __new__(cls, ip_arr, gd=None):
        obj = np.asarray(ip_arr).view(cls).astype(np.float64)
        obj.gd = gd
        return obj

In [459]:
class GDOptimizer:
    def __init__(self,lr):
        self.lr = lr
    def update(self,tup):
        w,b = tup
        if (w is None):
            return None,None
        return (w - self.lr*w.gd, b - self.lr*b.gd)
    def get_lr(self,):
        return self.lr
    def update_lr(self,lr):
        self.lr = lr

In [466]:
class LinearLayer:
    def __init__(self, n_in, n_out, b):
        self.batch_size = b
        self.w = jTensor(np.random.randn(n_in, n_out)*sqrt(2.0/n_in)) #He initialization
        self.b = jTensor(np.zeros((self.batch_size,n_out)))
    def __call__(self,x):
        return (np.dot(x,self.w) + self.b);
    def get_wb(self,):
        return (self.w,self.b)
    def backprop(self,x_in,x_out):
        x_in.gd = np.dot(x_out.gd,self.w.T)
        self.w.gd = np.dot(x_in.T,x_out.gd)
        self.b.gd = x_out.gd
    def update_wb(self,tup):
        w_new,b_new = tup
        self.w = w_new
        self.b = b_new
    def show_wb(self,):
        print("Weight Matrix")
        print(self.w)
        print("Bias Matrix")
        print(self.b)

In [467]:
class ReluLayer:
    def __init__(self):
        self.w = None
        self.b = None
    def __call__(self,x):
        return np.clip(x,a_min=0.0, a_max = None)
    def get_wb(self,):
        return (None,None)
    def backprop(self,x_in,x_out):
        temp = x_in
        temp[temp>0]=1.0
        temp[temp<=0]=0.0
        # dL/dx_in = dL/dxout * dxout/xin
        x_in.gd = (x_in > 0).astype(np.float64)*x_out.gd
    def update_wb(self,tup):
        w_new, b_new = None,None
    # def 

In [468]:
class MSELoss:
    def __call__(self,x,xt):
        return ((x-xt)**2)/len(x)
    def backprop(self,x,xt):
        x.gd = (2*(x-xt))/len(x) #dLoss/dx

In [479]:
class CreateModel:
    def __init__(self, layers_list, optimizer, loss_fn):
        self.layers_list = layers_list
        self.optimizer = optimizer
        self.loss = loss_fn
    def __call__(self,x_in):
        x_out = x_in
        for l in self.layers_list:
            x_out = l(x_out)
        return x_out
    def train(self,x_in,y_true, epochs=1):
        # assert len(x_in) == len(y_true) "Input vector and Output vector batch size should be equal!"
        for i in range(epochs):
            x_list=[]
            x_list.append(x_in)
            for l in self.layers_list:
                y = l(x_list[-1])
                # print(y.shape)
                x_list.append(y)
            print("Epoch: {}, Training Loss: {}, Optimizer lr: {}".format(i+1,np.mean(self.loss(x_list[-1],y_true)),self.optimizer.get_lr()))
            
            #Backprop
            self.loss.backprop(x_list[-1],y_true)
            for i in range(len(self.layers_list)):
                self.layers_list[-i-1].backprop(x_list[-i-2],x_list[-i-1])
            #Update
            for l in self.layers_list:
                l.update_wb(self.optimizer.update(l.get_wb()))
            x_list.clear()

In [480]:
layers_list = [
    LinearLayer(50,30,10),
    ReluLayer(),
    LinearLayer(30,40,10),
]

optim = GDOptimizer(lr = 1e-2)
loss = MSELoss()

In [481]:
model = CreateModel(layers_list,optim,loss)

In [482]:
X_in = jTensor(np.random.rand(10,50))
Y_true = jTensor(np.random.rand(10,40))

In [490]:
#Testing forward pass
y = model(X_in)
print(y.shape)
print(type(y))

(10, 40)
<class '__main__.jTensor'>


In [486]:
model.train(X_in, Y_true, 300)

Epoch: 1, Training Loss: 0.0021239223838360453, Optimizer lr: 0.01
Epoch: 2, Training Loss: 0.0021118771829258753, Optimizer lr: 0.01
Epoch: 3, Training Loss: 0.0020998290911941313, Optimizer lr: 0.01
Epoch: 4, Training Loss: 0.0020879974371941657, Optimizer lr: 0.01
Epoch: 5, Training Loss: 0.0020761337709716203, Optimizer lr: 0.01
Epoch: 6, Training Loss: 0.00206448296253606, Optimizer lr: 0.01
Epoch: 7, Training Loss: 0.0020524837955682, Optimizer lr: 0.01
Epoch: 8, Training Loss: 0.0020408477549354216, Optimizer lr: 0.01
Epoch: 9, Training Loss: 0.0020292198202974373, Optimizer lr: 0.01
Epoch: 10, Training Loss: 0.0020179847724203785, Optimizer lr: 0.01
Epoch: 11, Training Loss: 0.002006938109740224, Optimizer lr: 0.01
Epoch: 12, Training Loss: 0.0019952457132848865, Optimizer lr: 0.01
Epoch: 13, Training Loss: 0.001984077198078588, Optimizer lr: 0.01
Epoch: 14, Training Loss: 0.001972677923135854, Optimizer lr: 0.01
Epoch: 15, Training Loss: 0.0019612611763328443, Optimizer lr: 0.